In [105]:
# Processing content
from PIL import Image
import face_recognition
import numpy as np
import pickle

# Learning
from sklearn.linear_model import LinearRegression

from tqdm import tqdm
import glob, os
from io import BytesIO
import requests

In [73]:
def encode_image(img_path):
    """ Returns encoding
    """
    img = face_recognition.load_image_file(img_path)

    # Search for faces and encode
    face_locs = face_recognition.face_locations(img)
    if len(face_locs) > 0:
        return face_recognition.face_encodings(img, known_face_locations=face_locs)
    else:
        return [np.zeros((128))]
    
def encode_web_img(img_url):
    try:
        img_bytes = requests.get(img_url).content
        img = face_recognition.load_image_file(BytesIO(img_bytes))
    except Exception as e:
        print(e)
        return False

    # Search for faces and encode
    face_locs = face_recognition.face_locations(img)
    if len(face_locs) > 0:
        encodings = face_recognition.face_encodings(img, known_face_locations=face_locs)
        return encodings
    else:
        return [np.zeros((128))]


In [21]:
# Build encodings (samples)
datapath = './SCUT-FBP5500_v2/Images/*'
img_paths = glob.glob(datapath)
all_encodings = np.zeros((len(img_paths),128))

# Encode
for i, img_path in enumerate(tqdm(img_paths)):
    encodings = encode_image(impath)
    for enc in encodings:
        all_encodings[i,:] = enc
        
# Save
np.save("SCUT-encodings.npy", all_encodings)

100%|██████████| 5500/5500 [06:21<00:00, 14.40it/s]


In [37]:
# Build labels (targets)
with open('SCUT-FBP5500_v2/train_test_files/All_labels.txt') as f:
    # keys: filenames  //  values: ratings
    file_labels = {line.split(' ')[0]:float(line.split(' ')[1].strip()) for line in f.readlines()}
    
    labels = np.zeros((len(img_paths)))
    for i,path in enumerate(img_paths):
        _, filename = os.path.split(path) 
        labels[i] = file_labels[filename]
        
    np.save("SCUT-labels.npy", all_encodings)

In [54]:
file_encodings = {os.path.split(path):all_encodings[i,:] for i, path in enumerate(img_paths)}

In [53]:
reg = LinearRegression().fit(all_encodings, labels)
reg.score(all_encodings, labels)
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

0.6976930860192612

In [64]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
scores = cross_val_score(reg, all_encodings, labels, scoring=make_scorer(mean_absolute_error))
print("Avg Mean Absolute Error: {0}, Std: {1}".format(scores.mean(), scores.std()))
# https://scikit-learn.org/stable/modules/cross_validation.html

Avg Mean Absolute Error: 0.3043340797611177, Std: 0.006679926141786312


In [1]:
# Rate an image from a URL
myurl = input()
enc = encode_web_img(myurl)
print("Rating:", round(reg.predict(enc)[0]* 2,1))

NameError: name 'encode_web_img' is not defined